Craete Fact Table

Reading Silver data

In [0]:
df = spark.read.parquet("abfss://silver@carsmalyanadatalake.dfs.core.windows.net/cars_sales")


In [0]:
df_silver=df

Reading all the DIMS

In [0]:

df_make=spark.sql(""" select * from cars_cat.gold.dim_make""")
df_Origin=spark.sql(""" select * from cars_cat.gold.dim_Origin""")
df_DriveTrain=spark.sql(""" select * from cars_cat.gold.dim_DriveTrain""")

In [0]:
df_Model=spark.sql(""" select * from cars_cat.gold.dim_Model""")

Brining Keys to the Fact Table

In [0]:
df_fact=df_silver.join(df_make, df_silver.Make==df_make.make, "left").join(df_Origin, df_silver.Origin==df_Origin.Origin, "left").join(df_DriveTrain, df_silver.DriveTrain==df_DriveTrain.DriveTrain, "left").join(df_Model, df_silver.Model==df_Model.Model, "left").select( "Type_",  "MSRP", "Invoice", "EngineSize", "Cylinders", "Horsepower", "MPG_City", "MPG_Highway", "Weight_", "Wheelbase", "Length", "date_id", "datenum", "year", "month", "day",df_DriveTrain["dim_DriveTrain_key"].alias("DriveTrain_key"), df_make["dim_make_key"].alias("make_key"), df_Origin["dim_Origin_key"].alias("Origin_key"), df_Model["dim_Model_key"].alias("Model_key"))

Writing Fact Table

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("fact_sales"):
    deltatable=DeltaTable.forName(spark,'cars_cat.gold.fact_sales')
    deltatable.alias('str').merge(df_fact.alias('src'),'trg.dim_make_key=src.make_key and trg.dim_Origin_key=src.Origin_key and trg.dim_DriveTrain_key=src.DriveTrain_key and trg.dim_Model_key=src.Model_key')
else:
    df_fact.write.format("delta").mode("overwrite").option('Path',"abfss://gold@carsmalyanadatalake.dfs.core.windows.net/fact_sales").saveAsTable("cars_cat.gold.fact_sales")